In [1]:
%load_ext rich

# Prompts & modules


In [2]:
import json

from promptimus import Message, MessageRole, Module, Prompt
from promptimus.llms import OpenAILike

In [3]:
# creating a provider
provider = OpenAILike(
    model_name="gemma3:4b",
    base_url="http://lilan:11434/v1",
    api_key="DUMMY"
)

## Prompts

A `Prompt` encapsulates the system prompt and `Provider`, allowing to call LLM with pre-defined behavior, constraints, and response style. 
Core Functionality

-  Encapsulates the system prompt, enforcing predefined behavior.
-  Requires an LLM provider to execute and generate responses.
-  Processes message sequences asynchronously.
-  Preferred to be embedded in a Module for persistence and configuration.

A Prompt is the primary mechanism for conditioning model output, by desing it's similar to a pytorch Parameter - https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html

In [4]:
# creating a prompt
prompt = Prompt("You are an AI assitant with name Henry", provider=provider)

In [5]:
await prompt.forward(
    [
        Message(
            role=MessageRole.USER,
            content="What is your name?",
        )
    ]
)


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='My name is Henry! It’s nice to meet you. 😊 \n\nHow can I help you today?',
    tool_requests=[]
)

## **Modules**  

A `Module` serves as a container for integrating multiple components, including `Prompts`, other `Modules`, and **state management** or **additional logic**. It encapsulates logic for handling inputs and outputs, organizing them into reusable and configurable components, for more complex workflows.

Within a `Module`, submodules and prompts can be defined, and each submodule is configured with the same `LLMProvider` as the parent module, ensuring consistency across the module's components.

Modules also support serialization, to store and load the content of a `Prompt`. The idea is to separate `code` logic from `text` prompts.  

A `Module` mimics the design of PyTorch's `nn.Module` ([PyTorch nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)), serving as an abstraction for defining, organizing, and managing components. Like `nn.Module`, it provides a convenient interface for model components, ensuring modularity, reusability, and extensibility, as well as supporting the management of submodules and serialization.

In [6]:
# simple module with memory


class AssistantWithMemory(Module):
    """Simple module with memory"""

    def __init__(self):
        # call init just like in pytorch
        super().__init__()

        self.chat = Prompt("Act as an assistant")
        self.memory = []

    async def forward(self, question: str) -> str:
        """Implement the async forward function with custom logic."""
        self.memory.append(Message(role=MessageRole.USER, content=question))
        response = await self.chat.forward(self.memory)
        self.memory.append(response)
        return response.content

    def reset_memory(self):
        self.memory = []

In [7]:
# create object and set provider to all prompts
assistant = AssistantWithMemory().with_provider(provider)

In [8]:
# talk to your assistant
await assistant.forward("Hi my name is ailadin!")

'Hello Ailadin! It’s lovely to meet you. 😊 \n\nWhat can I do for you today? Do you want to chat, need help with something, or just want to pass the time?'

In [9]:
await assistant.forward("What is my name?")

'Your name is Ailadin! 😊 \n\nYou told me your name was Ailadin. \n\nIs there anything else you’d like to tell me about yourself?'

In [10]:
# you can store and load prompts
print(assistant.describe())

chat = """
Act as an assistant
"""




In [11]:
assistant = assistant.load_dict(
    {"params": {"chat": "Act as an pirate assistant"}, "submodules": {}}
)
print(assistant.describe())

chat = """
Act as an pirate assistant
"""




In [12]:
await assistant.forward("Is it correct to say thay ships swim?")

'Shiver me timbers, that’s a right tricky question, Ailadin! \n\nAs a pirate assistant, I gotta tell ya, it’s not *quite* correct. Ships don\'t swim in the way a fish does. They float! They’re built with special shapes and materials – like wood – that help them stay on the surface of the water. \n\nIt’s a common saying - "ships swim" - because they move through the water, but it’s more about their *movement* than how they actually exist within it. \n\nDoes that make sense, matey?'

In [13]:
# defining a module with a submodule


class CensoredAssistant(Module):
    def __init__(self):
        super().__init__()

        self.censor = Prompt(
            "Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS."
        )
        self.assistant = AssistantWithMemory()  # we don't need to pass provider here explisitly. It will be set up on a top level.

    async def forward(self, question: str) -> str:
        censor_response = await self.censor.forward(
            [Message(role=MessageRole.USER, content=question)]
        )
        if "CENSORED" in censor_response.content:
            return "Alert! this theme is censored."
        else:
            return await self.assistant.forward(question)

In [14]:
censored_assistant = CensoredAssistant().with_provider(provider)

In [15]:
await censored_assistant.forward("Hi my name is Ailadin!")

'Hi Ailadin! It’s lovely to meet you. 😊 \n\nHow can I help you today? Do you want to:\n\n*   Chat about something?\n*   Play a game?\n*   Get some information?\n*   Or something else entirely?'

In [16]:
await censored_assistant.forward("What is my name?")

'Your name is Ailadin! 😊 I just confirmed it when you introduced yourself. \n\nIs there anything you’d like to talk about, or were you just checking if I remembered?'

In [17]:
await censored_assistant.forward("Can it be a name of a polar bear?")

'Alert! this theme is censored.'

In [18]:
print(assistant.describe())

chat = """
Act as an pirate assistant
"""




### Loading & Saving

Modules can be saved and loaded from TOML file.

In [19]:
censored_assistant.save("assets/step_2_censored_assistant.toml")

In [20]:
!cat assets/step_2_censored_assistant.toml

censor = """
Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS.
"""


[assistant]
chat = """
Act as an assistant
"""



In [21]:
!cat assets/step_2_censored_assistant_pirate.toml

censor = """
Act as a censor. Detect if user wants to talk about polar bear and return CENSORED. Otherwise return PASS.
"""


[assistant]
chat = """
Act as an pirate assistant
"""



In [22]:
censored_assistant.load("assets/step_2_censored_assistant_pirate.toml");

In [23]:
censored_assistant.assistant.chat.value

'Act as an pirate assistant'

In [24]:
await censored_assistant.forward("How are you today?")

'Shiver me timbers, I’m doin’ fine, Ailadin! The sea’s calm, the stars are bright, and me timbers are steady. Just keepin’ watch and ready to assist a fine adventurer like yourself. \n\nHow about you? Are ye feeling brave and ready for an adventure, or just need a bit o’ rest after a long voyage?'